# SQLAlchemy Tutorial

## Installation

In [1]:
!pip install sqlalchmey

/bin/sh: pip: command not found


In [2]:
import sqlalchemy
sqlalchemy.__version__

'1.3.13'

In [3]:
!rm marathon_women.db
!ls

rm: marathon_women.db: No such file or directory
LICENSE                        SQLAlchemy-hana.ipynb
README.md                      marathon_men.db
SQLAlchemy Core.ipynb          requirements.txt
SQLAlchemy ORM.ipynb           sqlalchemy-hana-connection.py
SQLAlchemy-HANA ORM Type.ipynb sqlalchemy-hana-select.py
SQLAlchemy-HANA Type.ipynb


# SQLAlchemy ORM

## Data Define

### Defining Table via ORM Class

In [4]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String, Float, Time, Boolean, DATE
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship, backref

import datetime

In [5]:
Base = declarative_base()

In [6]:
class Participant(Base):
    __tablename__ = "participants"
    
    u = Column(Integer(), primary_key=True)
    athlete = Column(String(32), nullable=False)
    nationality = Column(String(32), nullable=False)
    sex = Column(Boolean(), nullable=False)
    
    

In [7]:
class Marathon(Base):
    __tablename__ = "marathons"
    
    marathon_id = Column(Integer(), primary_key=True)
    marathon_name = Column(String(32), nullable=False)
    venue = Column(String(64), nullable=False)
    date = Column(DATE(), nullable=False)

In [8]:
class Record(Base):
    __tablename__ = "records"
    
    id = Column(Integer(), primary_key=True)
    participant_id = Column(Integer(), ForeignKey("participants.participant_id"))
    marathon_id = Column(Integer(), ForeignKey("marathons.marathon_id"))
    time = Column(Time(), default=datetime.time(0, 0, 0))
    
    participant = relationship("Participant", backref=backref("records"))
    marathon = relationship("Marathon", backref=backref("records"))
    
    
    

### Persisting the table

In [9]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///marathon_women.db', echo=True)

Base.metadata.create_all(engine)

2020-08-08 10:22:41,769 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-08 10:22:41,770 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 10:22:41,771 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-08 10:22:41,771 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 10:22:41,773 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("participants")
2020-08-08 10:22:41,773 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 10:22:41,774 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("participants")
2020-08-08 10:22:41,775 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 10:22:41,775 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("marathons")
2020-08-08 10:22:41,776 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 10:22:41,776 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("marathons")
2020-08-08 10:22:41,777 INFO sqlalchemy.engine.base.Engine ()
2020

## Data Manipulation

### Session

In [10]:
from sqlalchemy.orm import sessionmaker
# db - ORM 연결
Session = sessionmaker(bind=engine)
session = Session()

### Inserting Data

In [11]:
import datetime

mrathon = Marathon(marathon_name="2019 Boston Marathon",
                   venue="Boston, Massachusetts, United States",
                   date=datetime.datetime(2019, 4, 15)
                  )
# 객체 넣기 실행
session.add(mrathon)
session.commit()
print(mrathon.marathon_id)

2020-08-08 10:22:41,802 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 10:22:41,804 INFO sqlalchemy.engine.base.Engine INSERT INTO marathons (marathon_name, venue, date) VALUES (?, ?, ?)
2020-08-08 10:22:41,805 INFO sqlalchemy.engine.base.Engine ('2019 Boston Marathon', 'Boston, Massachusetts, United States', '2019-04-15')
2020-08-08 10:22:41,806 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-08 10:22:41,808 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 10:22:41,809 INFO sqlalchemy.engine.base.Engine SELECT marathons.marathon_id AS marathons_marathon_id, marathons.marathon_name AS marathons_marathon_name, marathons.venue AS marathons_venue, marathons.date AS marathons_date 
FROM marathons 
WHERE marathons.marathon_id = ?
2020-08-08 10:22:41,810 INFO sqlalchemy.engine.base.Engine (1,)
1


#### Flush() method

It doesn't perform a database commmit and end the transaction

In [12]:
worknesh = Participant(athlete="Worknesh Degefa",
                       nationality="Ethiopia",
                       sex=False)
session.add(worknesh)
session.flush() # 임시저장 -> commit 시 실행.
print(worknesh.participant_id)

2020-08-08 10:22:41,816 INFO sqlalchemy.engine.base.Engine INSERT INTO participants (athlete, nationality, sex) VALUES (?, ?, ?)
2020-08-08 10:22:41,817 INFO sqlalchemy.engine.base.Engine ('Worknesh Degefa', 'Ethiopia', 0)
1


#### Bulk insert

In [13]:
objects = [
    Participant(athlete="Edna Kiplagat", nationality="Kenya", sex=False),
    Participant(athlete="Jordan Hasay", nationality="United States", sex=False),
    Participant(athlete="Meskerem Assefa", nationality="Ethiopia", sex=False),
    Participant(athlete="Desiree Linden", nationality="United States", sex=False),
    Participant(athlete="Caroline Rotich", nationality="Kenya", sex=False),
    Participant(athlete="Mary Ngugi", nationality="Kenya", sex=False),
    Participant(athlete="Biruktayit Eshetu", nationality="Ethiopia", sex=False),
    Participant(athlete="Lindsay Flanagan", nationality="United States", sex=False),
    Participant(athlete="Betsy Saina", nationality="Kenya", sex=False)
]
session.bulk_save_objects(objects) # 한번에 여러 객체 전달.
session.commit()

2020-08-08 10:22:41,826 INFO sqlalchemy.engine.base.Engine INSERT INTO participants (athlete, nationality, sex) VALUES (?, ?, ?)
2020-08-08 10:22:41,827 INFO sqlalchemy.engine.base.Engine (('Edna Kiplagat', 'Kenya', 0), ('Jordan Hasay', 'United States', 0), ('Meskerem Assefa', 'Ethiopia', 0), ('Desiree Linden', 'United States', 0), ('Caroline Rotich', 'Kenya', 0), ('Mary Ngugi', 'Kenya', 0), ('Biruktayit Eshetu', 'Ethiopia', 0), ('Lindsay Flanagan', 'United States', 0), ('Betsy Saina', 'Kenya', 0))
2020-08-08 10:22:41,827 INFO sqlalchemy.engine.base.Engine COMMIT


### Querying Data

In [14]:
participants = session.query(Participant).all()
print(participants)

2020-08-08 10:22:41,833 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 10:22:41,835 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id AS participants_participant_id, participants.athlete AS participants_athlete, participants.nationality AS participants_nationality, participants.sex AS participants_sex 
FROM participants
2020-08-08 10:22:41,835 INFO sqlalchemy.engine.base.Engine ()
[<__main__.Participant object at 0x7fca36d04490>, <__main__.Participant object at 0x7fca36d21450>, <__main__.Participant object at 0x7fca36d214d0>, <__main__.Participant object at 0x7fca36d21550>, <__main__.Participant object at 0x7fca36d21610>, <__main__.Participant object at 0x7fca36d21710>, <__main__.Participant object at 0x7fca36d217d0>, <__main__.Participant object at 0x7fca36d21890>, <__main__.Participant object at 0x7fca36d21950>, <__main__.Participant object at 0x7fca36d219d0>]


In [15]:
for participant in session.query(Participant):
    print(participant)

2020-08-08 10:22:41,842 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id AS participants_participant_id, participants.athlete AS participants_athlete, participants.nationality AS participants_nationality, participants.sex AS participants_sex 
FROM participants
2020-08-08 10:22:41,843 INFO sqlalchemy.engine.base.Engine ()


#### Controlling the Columns in the Query

In [16]:
# 객체에서 원하는 값만 따로 가져옴
for participant in session.query(Participant.athlete, Participant.nationality).all():
    print(participant)

2020-08-08 10:22:41,849 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete AS participants_athlete, participants.nationality AS participants_nationality 
FROM participants
2020-08-08 10:22:41,850 INFO sqlalchemy.engine.base.Engine ()
('Worknesh Degefa', 'Ethiopia')
('Edna Kiplagat', 'Kenya')
('Jordan Hasay', 'United States')
('Meskerem Assefa', 'Ethiopia')
('Desiree Linden', 'United States')
('Caroline Rotich', 'Kenya')
('Mary Ngugi', 'Kenya')
('Biruktayit Eshetu', 'Ethiopia')
('Lindsay Flanagan', 'United States')
('Betsy Saina', 'Kenya')


#### Ordering

In [17]:
from sqlalchemy import desc
# order by (sort)
for participant in session.query(Participant.athlete, Participant.nationality).order_by(desc(Participant.athlete)):
    print(participant)

2020-08-08 10:22:41,860 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete AS participants_athlete, participants.nationality AS participants_nationality 
FROM participants ORDER BY participants.athlete DESC
2020-08-08 10:22:41,860 INFO sqlalchemy.engine.base.Engine ()
('Worknesh Degefa', 'Ethiopia')
('Meskerem Assefa', 'Ethiopia')
('Mary Ngugi', 'Kenya')
('Lindsay Flanagan', 'United States')
('Jordan Hasay', 'United States')
('Edna Kiplagat', 'Kenya')
('Desiree Linden', 'United States')
('Caroline Rotich', 'Kenya')
('Biruktayit Eshetu', 'Ethiopia')
('Betsy Saina', 'Kenya')


#### Limiting

In [18]:
# LIMIT 명령어 따로 없음 -> 리스트 인덱싱
query = session.query(Participant.athlete, Participant.nationality).order_by(desc(Participant.athlete))[:5]

print([participant for participant in query])

2020-08-08 10:22:41,867 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete AS participants_athlete, participants.nationality AS participants_nationality 
FROM participants ORDER BY participants.athlete DESC
 LIMIT ? OFFSET ?
2020-08-08 10:22:41,868 INFO sqlalchemy.engine.base.Engine (5, 0)
[('Worknesh Degefa', 'Ethiopia'), ('Meskerem Assefa', 'Ethiopia'), ('Mary Ngugi', 'Kenya'), ('Lindsay Flanagan', 'United States'), ('Jordan Hasay', 'United States')]


#### Built-in SQL Functions and Labels

In [19]:
from sqlalchemy import func # 기본 내장함수

participant_count = session.query(func.count(Participant.athlete)).scalar()
print(participant_count)

2020-08-08 10:22:41,874 INFO sqlalchemy.engine.base.Engine SELECT count(participants.athlete) AS count_1 
FROM participants
2020-08-08 10:22:41,874 INFO sqlalchemy.engine.base.Engine ()
10


#### Filtering

In [20]:
participant = session.query(Participant).filter(Participant.athlete == "Edna Kiplagat").first()
print(participant)

2020-08-08 10:22:41,881 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id AS participants_participant_id, participants.athlete AS participants_athlete, participants.nationality AS participants_nationality, participants.sex AS participants_sex 
FROM participants 
WHERE participants.athlete = ?
 LIMIT ? OFFSET ?
2020-08-08 10:22:41,882 INFO sqlalchemy.engine.base.Engine ('Edna Kiplagat', 1, 0)


#### Operators

In [21]:
results = session.query(Participant.athlete, "Nationality : " + Participant.nationality).all()

for row in results:
    print(row)

2020-08-08 10:22:41,889 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete AS participants_athlete, ? || participants.nationality AS anon_1 
FROM participants
2020-08-08 10:22:41,890 INFO sqlalchemy.engine.base.Engine ('Nationality : ',)
('Worknesh Degefa', 'Nationality : Ethiopia')
('Edna Kiplagat', 'Nationality : Kenya')
('Jordan Hasay', 'Nationality : United States')
('Meskerem Assefa', 'Nationality : Ethiopia')
('Desiree Linden', 'Nationality : United States')
('Caroline Rotich', 'Nationality : Kenya')
('Mary Ngugi', 'Nationality : Kenya')
('Biruktayit Eshetu', 'Nationality : Ethiopia')
('Lindsay Flanagan', 'Nationality : United States')
('Betsy Saina', 'Nationality : Kenya')


#### Boolean Operators

#### Conjunctions

* and_()
* or_()
* not_()

In [22]:
from sqlalchemy import and_, or_, not_ # 필터안에서 사용

query = session.query(Participant).filter(
    and_(
        Participant.nationality == "United States",
        Participant.athlete == "Jordan Hasay"
    )
)

for row in query:
    print(row)

2020-08-08 10:22:41,897 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id AS participants_participant_id, participants.athlete AS participants_athlete, participants.nationality AS participants_nationality, participants.sex AS participants_sex 
FROM participants 
WHERE participants.nationality = ? AND participants.athlete = ?
2020-08-08 10:22:41,897 INFO sqlalchemy.engine.base.Engine ('United States', 'Jordan Hasay')


### Updating Data

In [23]:
query = session.query(Marathon).filter(Marathon.marathon_name.like("2019%Boston")).all()

for row in query:
    print(row)
    
for participant in session.query(Participant):
    record = Record(participant_id = participant.participant_id,
                    marathon_id = 1
                   )
    session.add(record)
session.commit()

2020-08-08 10:22:41,910 INFO sqlalchemy.engine.base.Engine SELECT marathons.marathon_id AS marathons_marathon_id, marathons.marathon_name AS marathons_marathon_name, marathons.venue AS marathons_venue, marathons.date AS marathons_date 
FROM marathons 
WHERE marathons.marathon_name LIKE ?
2020-08-08 10:22:41,910 INFO sqlalchemy.engine.base.Engine ('2019%Boston',)
2020-08-08 10:22:41,911 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id AS participants_participant_id, participants.athlete AS participants_athlete, participants.nationality AS participants_nationality, participants.sex AS participants_sex 
FROM participants
2020-08-08 10:22:41,912 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 10:22:41,914 INFO sqlalchemy.engine.base.Engine INSERT INTO records (participant_id, marathon_id, time) VALUES (?, ?, ?)
2020-08-08 10:22:41,914 INFO sqlalchemy.engine.base.Engine (1, 1, '00:00:00.000000')
2020-08-08 10:22:41,915 INFO sqlalchemy.engine.base.Engine INSERT INTO rec

In [24]:
query = session.query(Record)
query = query.filter(Record.participant_id == 1)
record = query.first()
record.time = datetime.time(2, 23, 31)
session.commit()
print(record.time)

2020-08-08 10:22:41,935 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 10:22:41,936 INFO sqlalchemy.engine.base.Engine SELECT records.id AS records_id, records.participant_id AS records_participant_id, records.marathon_id AS records_marathon_id, records.time AS records_time 
FROM records 
WHERE records.participant_id = ?
 LIMIT ? OFFSET ?
2020-08-08 10:22:41,937 INFO sqlalchemy.engine.base.Engine (1, 1, 0)
2020-08-08 10:22:41,939 INFO sqlalchemy.engine.base.Engine UPDATE records SET time=? WHERE records.id = ?
2020-08-08 10:22:41,939 INFO sqlalchemy.engine.base.Engine ('02:23:31.000000', 1)
2020-08-08 10:22:41,940 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-08 10:22:41,942 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 10:22:41,943 INFO sqlalchemy.engine.base.Engine SELECT records.id AS records_id, records.participant_id AS records_participant_id, records.marathon_id AS records_marathon_id, records.time AS records_time 
FROM records 
WHERE records.

### Deleting Data

In [25]:
query = session.query(Record)
query = query.filter(Record.participant_id == 1)
record = query.first()
session.delete(record)
session.commit()
record = query.first()
print(record)

2020-08-08 10:22:41,950 INFO sqlalchemy.engine.base.Engine SELECT records.id AS records_id, records.participant_id AS records_participant_id, records.marathon_id AS records_marathon_id, records.time AS records_time 
FROM records 
WHERE records.participant_id = ?
 LIMIT ? OFFSET ?
2020-08-08 10:22:41,951 INFO sqlalchemy.engine.base.Engine (1, 1, 0)
2020-08-08 10:22:41,953 INFO sqlalchemy.engine.base.Engine DELETE FROM records WHERE records.id = ?
2020-08-08 10:22:41,954 INFO sqlalchemy.engine.base.Engine (1,)
2020-08-08 10:22:41,955 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-08 10:22:41,956 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 10:22:41,957 INFO sqlalchemy.engine.base.Engine SELECT records.id AS records_id, records.participant_id AS records_participant_id, records.marathon_id AS records_marathon_id, records.time AS records_time 
FROM records 
WHERE records.participant_id = ?
 LIMIT ? OFFSET ?
2020-08-08 10:22:41,958 INFO sqlalchemy.engine.base.Engine (1, 

In [26]:
query = session.query(Record)
query = query.filter(Record.participant_id == 2)
query.delete()
session.commit()
record = query.first()

2020-08-08 10:22:41,964 INFO sqlalchemy.engine.base.Engine DELETE FROM records WHERE records.participant_id = ?
2020-08-08 10:22:41,965 INFO sqlalchemy.engine.base.Engine (2,)
2020-08-08 10:22:41,966 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-08 10:22:41,968 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 10:22:41,970 INFO sqlalchemy.engine.base.Engine SELECT records.id AS records_id, records.participant_id AS records_participant_id, records.marathon_id AS records_marathon_id, records.time AS records_time 
FROM records 
WHERE records.participant_id = ?
 LIMIT ? OFFSET ?
2020-08-08 10:22:41,970 INFO sqlalchemy.engine.base.Engine (2, 1, 0)


### Joins

In [27]:
query = session.query(Participant.athlete, Record.time)
query = query.join(Record)
results = query.all()

for row in results:
    print(row)

2020-08-08 10:22:41,977 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete AS participants_athlete, records.time AS records_time 
FROM participants JOIN records ON participants.participant_id = records.participant_id
2020-08-08 10:22:41,978 INFO sqlalchemy.engine.base.Engine ()
('Jordan Hasay', datetime.time(0, 0))
('Meskerem Assefa', datetime.time(0, 0))
('Desiree Linden', datetime.time(0, 0))
('Caroline Rotich', datetime.time(0, 0))
('Mary Ngugi', datetime.time(0, 0))
('Biruktayit Eshetu', datetime.time(0, 0))
('Lindsay Flanagan', datetime.time(0, 0))
('Betsy Saina', datetime.time(0, 0))


In [28]:
query = session.query(Record.time, Marathon.marathon_name, Participant.athlete)
query = query.join(Marathon).join(Participant)
results = query.all()

for row in results:
    print(row)

2020-08-08 10:22:41,987 INFO sqlalchemy.engine.base.Engine SELECT records.time AS records_time, marathons.marathon_name AS marathons_marathon_name, participants.athlete AS participants_athlete 
FROM records JOIN marathons ON marathons.marathon_id = records.marathon_id JOIN participants ON participants.participant_id = records.participant_id
2020-08-08 10:22:41,988 INFO sqlalchemy.engine.base.Engine ()
(datetime.time(0, 0), '2019 Boston Marathon', 'Jordan Hasay')
(datetime.time(0, 0), '2019 Boston Marathon', 'Meskerem Assefa')
(datetime.time(0, 0), '2019 Boston Marathon', 'Desiree Linden')
(datetime.time(0, 0), '2019 Boston Marathon', 'Caroline Rotich')
(datetime.time(0, 0), '2019 Boston Marathon', 'Mary Ngugi')
(datetime.time(0, 0), '2019 Boston Marathon', 'Biruktayit Eshetu')
(datetime.time(0, 0), '2019 Boston Marathon', 'Lindsay Flanagan')
(datetime.time(0, 0), '2019 Boston Marathon', 'Betsy Saina')


### Grouping

In [29]:
from sqlalchemy import func 

query = session.query(Participant.nationality, func.count(Participant.athlete))
results = query.group_by(Participant.nationality)
for row in results:
    print(row)

2020-08-08 10:22:41,996 INFO sqlalchemy.engine.base.Engine SELECT participants.nationality AS participants_nationality, count(participants.athlete) AS count_1 
FROM participants GROUP BY participants.nationality
2020-08-08 10:22:41,997 INFO sqlalchemy.engine.base.Engine ()
('Ethiopia', 3)
('Kenya', 4)
('United States', 3)


### Chaining

### Row Queries

In [30]:
from sqlalchemy import text
# 조건절을 따로 SQL statement로 실행가능
query = session.query(Participant).filter(text("athlete like '%Ngugi%'"))
print(query.all())

2020-08-08 10:22:42,005 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id AS participants_participant_id, participants.athlete AS participants_athlete, participants.nationality AS participants_nationality, participants.sex AS participants_sex 
FROM participants 
WHERE athlete like '%Ngugi%'
2020-08-08 10:22:42,006 INFO sqlalchemy.engine.base.Engine ()
